<a href="https://colab.research.google.com/github/itchy-squid/AdventOfCode/blob/2025-1/2025/2025.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import files
fread = lambda path: open(path,'r').read()

In [53]:
#@title Day 1-1: Secret Entrance
inputs = fread('25-1.txt').splitlines()

curr = 50
pwd = 0

for i in inputs:
  curr = (curr + (-1 if i[0] == 'L' else 1) * int(i[1:])) % 100
  pwd = pwd + (curr == 0)

print("pwd:", pwd)

pwd: 997


In [67]:
#@title Day 1-2: Secret Entrance
import math

inputs = fread('25-1.txt').splitlines()

curr = 50
part2 = 0

for i in inputs:
  mutate = lambda p : (100-p)%100 if i[0] == 'L' else p%100
  curr = mutate(curr) + int(i[1:])
  part2 = part2 + math.floor(curr / 100)
  curr = mutate(curr)

print("part2:", part2)

pwd: 5978
